# Aufgabe
Textdatei einlesen und diese dann soweit "aufräumen" dass die Wörter gezählt werden können


In [1]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Franky") \
      .getOrCreate() 

# Einlesen von Dateien in Spark, Datei Typ einfach Anpassen
frank=spark.read.text("frankenstein.txt")
frank.show(truncate=False)
# Schema anzeigen lassen
frank.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/18 14:06:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------------------------------------------------------------------------+
|value                                                                    |
+-------------------------------------------------------------------------+
|                                                                         |
|Project Gutenberg's Frankenstein, by Mary Wollstonecraft (Godwin) Shelley|
|                                                                         |
|This eBook is for the use of anyone anywhere at no cost and with         |
|almost no restrictions whatsoever.  You may copy it, give it away or     |
|re-use it under the terms of the Project Gutenberg License included      |
|with this eBook or online at www.gutenberg.net                           |
|                                                                         |
|                                                                         |
|Title: Frankenstein                                                      |
|       or T

In [3]:
#Überprüfen was für dtypes drin sind
print(frank.dtypes)

[('value', 'string')]


Der erste Split hier kann übersprungen werden wenn nacher noch ein ein Regex split kommt

In [52]:
import pyspark.sql.functions as F
# select selektiert eine oder mehrere Spalten, hier eben nur die 1 Spalte "value"
# alias gibt dem Ergebnis (der selektierten Spalte) einen Namen (sonst waere es eher unhandlich)
# die Funktion split nimmt uebrigens eine REGEXP ... hier ist sie halt super-einfach> ein space
lines = frank.select(F.split(frank.value, " ").alias("Zeile"))
lines.show(10, truncate=100)
# das Schema: das Split hat ein Array generiert
lines.printSchema()


+----------------------------------------------------------------------------------+
|                                                                             Zeile|
+----------------------------------------------------------------------------------+
|                                                                                []|
|[Project, Gutenberg's, Frankenstein,, by, Mary, Wollstonecraft, (Godwin), Shelley]|
|                                                                                []|
|   [This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]|
|[almost, no, restrictions, whatsoever., , You, may, copy, it,, give, it, away, or]|
|   [re-use, it, under, the, terms, of, the, Project, Gutenberg, License, included]|
|                            [with, this, eBook, or, online, at, www.gutenberg.net]|
|                                                                                []|
|                                                                

7834

In [6]:
lines.show(10, truncate=False)

+----------------------------------------------------------------------------------+
|Zeile                                                                             |
+----------------------------------------------------------------------------------+
|[]                                                                                |
|[Project, Gutenberg's, Frankenstein,, by, Mary, Wollstonecraft, (Godwin), Shelley]|
|[]                                                                                |
|[This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]   |
|[almost, no, restrictions, whatsoever., , You, may, copy, it,, give, it, away, or]|
|[re-use, it, under, the, terms, of, the, Project, Gutenberg, License, included]   |
|[with, this, eBook, or, online, at, www.gutenberg.net]                            |
|[]                                                                                |
|[]                                                              

In [53]:
# Regex um Wörter zu finden welche mit einem Buchstaben anfangen, ob Groß oder Klein, 
lines = frank.select(F.split(frank.value, "[^a-zA-Z]").alias("Zeile"))
lines.show(10,truncate=False)


+--------------------------------------------------------------------------------------+
|Zeile                                                                                 |
+--------------------------------------------------------------------------------------+
|[]                                                                                    |
|[Project, Gutenberg, s, Frankenstein, , by, Mary, Wollstonecraft, , Godwin, , Shelley]|
|[]                                                                                    |
|[This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]       |
|[almost, no, restrictions, whatsoever, , , You, may, copy, it, , give, it, away, or]  |
|[re, use, it, under, the, terms, of, the, Project, Gutenberg, License, included]      |
|[with, this, eBook, or, online, at, www, gutenberg, net]                              |
|[]                                                                                    |
|[]                  

7834

In [12]:
#Expoden der Wörter

# Explode erstellt aus jedem Objekt aus einem Array eine neue Zeile
words = lines.select(F.explode(F.col("Zeile")).alias("Word"))
words.show()

+--------------+
|          Word|
+--------------+
|              |
|       Project|
|     Gutenberg|
|             s|
|  Frankenstein|
|              |
|            by|
|          Mary|
|Wollstonecraft|
|              |
|        Godwin|
|              |
|       Shelley|
|              |
|          This|
|         eBook|
|            is|
|           for|
|           the|
|           use|
+--------------+
only showing top 20 rows



In [28]:
#jetzt wo der Explode gemacht wurde, die wörter aus designgründen alle klein schreiben
words_lower=words.select(F.lower(F.col("Word")).alias("lower_words"))
words_lower.show()

+--------------+
|   lower_words|
+--------------+
|              |
|       project|
|     gutenberg|
|             s|
|  frankenstein|
|              |
|            by|
|          mary|
|wollstonecraft|
|              |
|        godwin|
|              |
|       shelley|
|              |
|          this|
|         ebook|
|            is|
|           for|
|           the|
|           use|
+--------------+
only showing top 20 rows



In [29]:
#jetzt leere Zeilen entfernen
words_clean= words_lower.filter((F.col("lower_words")!="").alias("wörter"))
words_clean.show()

+--------------+
|   lower_words|
+--------------+
|       project|
|     gutenberg|
|             s|
|  frankenstein|
|            by|
|          mary|
|wollstonecraft|
|        godwin|
|       shelley|
|          this|
|         ebook|
|            is|
|           for|
|           the|
|           use|
|            of|
|        anyone|
|      anywhere|
|            at|
|            no|
+--------------+
only showing top 20 rows



In [34]:
#Jetzt müssen noch einzelne Buchstaben wie das allein stehende s entfernt werden, a und i bleiben hier jedoch drinnen weil Englischer Text
words_final=words_clean.select(F.regexp_extract(F.col("lower_words"),"[a-z]{2,}|a|i",0).alias("Final"))
words_final.show()

+--------------+
|         Final|
+--------------+
|       project|
|     gutenberg|
|              |
|  frankenstein|
|            by|
|          mary|
|wollstonecraft|
|        godwin|
|       shelley|
|          this|
|         ebook|
|            is|
|           for|
|           the|
|           use|
|            of|
|        anyone|
|      anywhere|
|            at|
|            no|
+--------------+
only showing top 20 rows



In [36]:
#jetzt nochmal die leeren zeilen entfernen
words_final2= words_final.filter(F.col("Final")!="")
words_final2.show()

+--------------+
|         Final|
+--------------+
|       project|
|     gutenberg|
|  frankenstein|
|            by|
|          mary|
|wollstonecraft|
|        godwin|
|       shelley|
|          this|
|         ebook|
|            is|
|           for|
|           the|
|           use|
|            of|
|        anyone|
|      anywhere|
|            at|
|            no|
|          cost|
+--------------+
only showing top 20 rows



In [39]:
#jetzt kann man die wörter zählen
word_count=words_final2.count()
print(word_count)

78236


In [41]:
#jetzt wie viel eindeutige Wörter
word_count_distinct=words_final2.distinct().count()
print(f'Die Datei hat {word_count_distinct} einzelne Wörter')

Die Datei hat 7256 einzelne Wörter


In [51]:
#jetzt noch alle befehel in einen Codeblock
test= (
    spark.read.text("frankenstein.txt")
    .select(F.split(F.col("value"), "[^a-zA-Z]").alias("Zeile"))
    .select(F.explode(F.col("Zeile")).alias("Word"))
    .select(F.lower(F.col("Word")).alias("lower_words"))
    .filter((F.col("lower_words")!="").alias("wörter"))
    .select(F.regexp_extract(F.col("lower_words"),"[a-z]{2,}|a|i",0).alias("Final"))
    .filter(F.col("Final")!="")
    .distinct().count()
)
print(test)
    
    

7256
